In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [ ]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, create_spotify_playlist, get_track_list, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artists
from plan import validate_plan

tools = [get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, search_tool, find_similar_artists, get_track_list]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Bind Tools to Model

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=0.8)
llm_with_tools = llm.bind_tools(tools, strict=True, parallel_tool_calls=False)

# First Message

In [8]:
from langchain_core.messages import HumanMessage
from prompts import Prompts

human_message = HumanMessage(Prompts.SPOTIFY)
ai_tool_call_message = llm_with_tools.invoke([human_message])

2024-11-12 23:50:01,348 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
ai_tool_call_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  validate_plan (call_Q1MVgHJU2J6qSNyUkIlitq39)
 Call ID: call_Q1MVgHJU2J6qSNyUkIlitq39
  Args:
    plan: {'steps': [{'name': 'get_track_list', 'description': "Retrieve the track list for the 'New Rock and Blues' playlist to identify existing artists and tracks.", 'success_criteria': "Successfully obtained a list of tracks and artists from the 'New Rock and Blues' playlist.", 'tool': 'functions.get_track_list', 'action': "Retrieve the track list for the 'New Rock and Blues' playlist using the function tool."}, {'name': 'get_artists_from_playlist', 'description': "Get the list of unique artists from 'New Rock and Blues' playlist.", 'success_criteria': 'Successfully obtained the list of unique artists from the playlist.', 'tool': 'functions.get_artists_from_playlist', 'action': "Use the function tool to get unique artists from the 'New Rock and Blues' playlist."}, {'name': 'find_similar_artists',

In [10]:
import json
plan = ai_tool_call_message.additional_kwargs["tool_calls"][0]["function"]["arguments"]
json.loads(plan)

{'plan': {'steps': [{'name': 'get_track_list',
    'description': "Retrieve the track list for the 'New Rock and Blues' playlist to identify existing artists and tracks.",
    'success_criteria': "Successfully obtained a list of tracks and artists from the 'New Rock and Blues' playlist.",
    'tool': 'functions.get_track_list',
    'action': "Retrieve the track list for the 'New Rock and Blues' playlist using the function tool."},
   {'name': 'get_artists_from_playlist',
    'description': "Get the list of unique artists from 'New Rock and Blues' playlist.",
    'success_criteria': 'Successfully obtained the list of unique artists from the playlist.',
    'tool': 'functions.get_artists_from_playlist',
    'action': "Use the function tool to get unique artists from the 'New Rock and Blues' playlist."},
   {'name': 'find_similar_artists',
    'description': "Find similar artists for each artist in the 'New Rock and Blues' playlist.",
    'success_criteria': 'Successfully found 3-4 simila

# chat_prompt_template holds all messages (Human, AI, Tool)

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template: ChatPromptTemplate = human_message + ai_tool_call_message
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        considering tempo, energy, a

# Check for Tools in Message

In [ ]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0:
    raise ValueError
else:
    print(tools_in_ai_message)

# Tool Call (should be validate_plan())

In [12]:
response = tool_node.invoke({"messages": messages})
tool_message = response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: validate_plan

true


In [13]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        considering tempo, energy, a

# Send tool result to LLM

In [14]:
ai_tool_call_message = llm_with_tools.invoke(chat_prompt_template.format_messages())
ai_tool_call_message.pretty_print()

2024-11-12 23:51:21,575 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

The plan has been validated and is ready for execution. We'll follow the steps as outlined to build a Spotify playlist with tracks that match the vibe and genres of your 'New Rock and Blues' playlist, while adhering to the specified rules. Let's begin with Step 1.

### Step 1: Retrieve the Track List

**Objective:** Retrieve the track list for the 'New Rock and Blues' playlist to identify existing artists and tracks.

Let's proceed to retrieve the track list from the 'New Rock and Blues' playlist.
Tool Calls:
  get_playlists (call_UFoXZzxg5t85fjX7kv0AHHcq)
 Call ID: call_UFoXZzxg5t85fjX7kv0AHHcq
  Args:


# Tool Call (should be get_playlists())

In [15]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()
response = tool_node.invoke({"messages": messages})

In [17]:
from langchain_core.messages import ToolMessage

playlist_tool_message: ToolMessage = response["messages"][0]
playlist_tool_message.pretty_print()

================================= Tool Message =================================
Name: get_playlists

{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "descripti

In [ ]:
# We need to check that is JSON serializable
# json.loads(playlist_tool_message.content)

# Format Message List for LLM

In [18]:
chat_prompt_template += playlist_tool_message
messages = chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [19]:
ai_tool_call_message = llm_with_tools.invoke(messages)
ai_tool_call_message.pretty_print()

2024-11-12 23:52:35,551 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_track_list (call_Uz1wDOnosiuMV9Fz5ZAvFC0W)
 Call ID: call_Uz1wDOnosiuMV9Fz5ZAvFC0W
  Args:
    playlist_id: spotify:playlist:01tk0aitEuGK0ajWCkzdKc


In [20]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [21]:
response = tool_node.invoke({"messages": messages})
track_list_tool_message: ToolMessage = response["messages"][0]

# Decode AI Tool Call message

In [22]:
track_list_tool_message.pretty_print()

================================= Tool Message =================================
Name: get_track_list

{"tracks": [{"id": "59oXRNRnmmTbwgkZkV7l4g", "uri": "spotify:track:59oXRNRnmmTbwgkZkV7l4g", "name": "Low Down Rolling Stone", "artists": ["Gary Clark Jr."], "album": "This Land", "duration_ms": 258200, "explicit": false, "popularity": 46}, {"id": "2TmVlVk2t6d9MoqLNasfgS", "uri": "spotify:track:2TmVlVk2t6d9MoqLNasfgS", "name": "Take Me Away", "artists": ["Ayron Jones"], "album": "Child Of The State", "duration_ms": 246932, "explicit": true, "popularity": 41}, {"id": "5UMD1Iz7yyVK8Q5FLsVt3q", "uri": "spotify:track:5UMD1Iz7yyVK8Q5FLsVt3q", "name": "Walk On Water", "artists": ["Thirty Seconds To Mars"], "album": "Walk On Water", "duration_ms": 188227, "explicit": false, "popularity": 0}, {"id": "0Tr5G2mE56eLUGvCaXRM8I", "uri": "spotify:track:0Tr5G2mE56eLUGvCaXRM8I", "name": "No Good", "artists": ["KALEO"], "album": "A/B", "duration_ms": 234240, "explicit": false, "popularity": 64}, {"id":

In [ ]:
# json.loads(track_list_tool_message.content)

# Add Tool message containing artists to list of messages

In [23]:
chat_prompt_template += track_list_tool_message
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        considering tempo, energy, a

# Send Tool Result to LLM

In [24]:
response = llm_with_tools.invoke(messages)

2024-11-12 23:53:03,244 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI Tool Call (find_similar_artist())

In [25]:
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_artists_from_playlist (call_nfohw6df15f48ZEu0bcpnYlz)
 Call ID: call_nfohw6df15f48ZEu0bcpnYlz
  Args:
    playlist_id: spotify:playlist:01tk0aitEuGK0ajWCkzdKc


In [26]:
chat_prompt_template += response
messages = chat_prompt_template.format_messages()

In [ ]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
print(tools_in_ai_message)
if len(tools_in_ai_message) == 0:
    raise ValueError

In [ ]:
messages

# Tool Call (find_similar_artists())

In [27]:
response = tool_node.invoke({"messages": messages})

In [31]:
tool_message = response["messages"][0]
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\n    Create, validate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n    **Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.\n\n    **Rules:**\n\n    1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n    2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n    3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n    4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.\n\n    5. **Recommend 3-4 tracks for each new artist.\n\n    6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, \n        considering tempo, energy, a

In [32]:
response = llm_with_tools.invoke(messages)

2024-11-12 23:58:36,303 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [34]:
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  find_similar_artist (call_yoN81jKc0I38ydGbvEbR2Dyi)
 Call ID: call_yoN81jKc0I38ydGbvEbR2Dyi
  Args:
    artist: spotify:artist:01aC2ikO4Xgb2LUpf9JfKp
  find_similar_artist (call_fNW7b3mIayT0A9x3yr9A3rYK)
 Call ID: call_fNW7b3mIayT0A9x3yr9A3rYK
  Args:
    artist: spotify:artist:1iEaqWaYpKo9x0OrEq7Q7z
  find_similar_artist (call_Gabf63wcS8HPdRwNAqDnabjr)
 Call ID: call_Gabf63wcS8HPdRwNAqDnabjr
  Args:
    artist: spotify:artist:0RqtSIYZmd4fiBKVFqyIqD
  find_similar_artist (call_zvOL6lNyUyQd82vxv1F5fC6o)
 Call ID: call_zvOL6lNyUyQd82vxv1F5fC6o
  Args:
    artist: spotify:artist:7jdFEYD2LTYjfwxOdlVjmc
  find_similar_artist (call_rnf6BG0m4nxjjUu41dd5genx)
 Call ID: call_rnf6BG0m4nxjjUu41dd5genx
  Args:
    artist: spotify:artist:7Ln80lUS6He07XvHI8qqHH


In [ ]:
from typing import List


similar_artists_tool_messages: List[ToolMessage] = response["messages"]
[m.pretty_print() for m in similar_artists_tool_messages]

In [ ]:
chat_prompt_template += similar_artists_tool_messages
messages = chat_prompt_template.format_messages()
messages

In [ ]:
response = llm_with_tools.invoke(messages)

In [ ]:
response.pretty_print()

In [ ]:
chat_prompt_template += response
chat_prompt_template.format_messages()

In [ ]:
response = tool_node.invoke({"messages": [response]})

In [ ]:
response